## CONFIG

In [103]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup



from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [ ]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [105]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [106]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

## METODI

In [149]:
import json

def estrai_categorie_rilevanti(testo, categorie, llm):
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e dei loro Menu, Piatti, Ingredienti e Tecniche. 
    
    Voglio che analizzi il testo e mi restituisca un JSON con:
    - Una chiave per ogni categoria rilevante tra queste: {', '.join(categorie)}
    - Un valore float tra 0 e 100 che indica quanto è rilevante quella categoria nel testo

    Se una categoria non è rilevante, assegna 0. Restituisci solo il JSON, senza markdown o decoratori.

    Testo:
    {testo}
    """

    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        punteggi = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON:", content)
        return []

    # Filtra solo le categorie con punteggio > 0
    categorie_rilevanti = [categoria for categoria, punteggio in punteggi.items() if punteggio > 10]

    return categorie_rilevanti


In [107]:
def estrai_valorizzazioni(testo, categoria, llm, driver):
    # Genera un prompt basato sulle entità filtrate
    
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti e Tecniche. 
    Il testo che segue potrebbe contenere informazioni pertinenti a questa categoria: {categoria}
    Voglio che tu estragga lr possibili istanze di questa categoria, per ogni istanza recupera:
        - NOME dell'istanza
        - eventuali PARAMETRI associati all'istanza
    Attento che i nomi delle chiavi devono essere identiche a quelle fornite.
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.

    Testo:
    {testo}
    """

    # Chiamata al modello LLM per ottenere la valorizzazione delle entità
    response = llm.invoke(prompt)
    content = response.content.strip()

    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()

    try:
        valorizzazioni = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle valorizzazioni:", content)
        valorizzazioni = {}
    
    return valorizzazioni


In [108]:
def carica_piatti(file_json: str):
    """
    Carica i piatti e i numeri associati come nodi nel database Neo4j.
    
    Args:
        file_json (str): Percorso del file JSON contenente i piatti e i numeri.
    """
    
    # Carica il file JSON
    with open(file_json, 'r', encoding='utf-8') as f:
        piatti_data = json.load(f)
    
    with driver.session() as session:
        for piatto, numero in piatti_data.items():
            # Decodifica i caratteri speciali (ad esempio per trattare le sequenze \u00e0)
            piatto_decodificato = html.unescape(piatto)
            
            # Crea un nodo per ciascun piatto
            query = """
            MERGE (p:Piatto {nome: $nome})
            SET p.numero = $numero
            """
            session.run(query, nome=piatto_decodificato, numero=numero)
        
        print("Piatti caricati correttamente in Neo4j.")

In [130]:
def carica_entita_su_neo4j(categoria, entities, driver):
    """
    Carica le entità filtrate come nodi nel database Neo4j.
    
    Args:
        categoria (str): La categoria/label da utilizzare per i nodi.
        entities (list): Lista di entità da caricare.
        driver: Connessione al database Neo4j.
    """
    
    with driver.session() as session:
        if not entities:  # Controlla se la lista di entità è vuota
            print(f"Nessuna entità trovata per la categoria {categoria}")
            return
            
        for entity in entities:
            # Controlla se l'entità ha la proprietà 'NOME'
            if 'NOME' not in entity or not entity['NOME']:
                print(f"Entità senza nome trovata in categoria {categoria}, skip")
                continue
            
            # Assicurati che il nome sia decodificato per gestire caratteri speciali
            nome_decodificato = html.unescape(entity['NOME'])
            
            # Prepara le proprietà base
            properties = {'nome': nome_decodificato}
            
            # Aggiungi i parametri se presenti
            if 'PARAMETRI' in entity and entity['PARAMETRI']:
                # Se PARAMETRI è una lista di stringhe
                if isinstance(entity['PARAMETRI'], list):
                    # Convertire la lista in una stringa con separatore
                    properties['parametri'] = ', '.join(entity['PARAMETRI'])
                    # Opzionalmente, aggiungi anche i singoli elementi numerati
                    for i, param in enumerate(entity['PARAMETRI']):
                        properties[f'parametro_{i+1}'] = param
                # Se PARAMETRI è un dizionario
                elif isinstance(entity['PARAMETRI'], dict):
                    for key, value in entity['PARAMETRI'].items():
                        properties[key.lower()] = value
                # Se PARAMETRI è una stringa singola
                elif isinstance(entity['PARAMETRI'], str):
                    properties['parametri'] = entity['PARAMETRI']
            
            # Crea un nodo per ciascuna entità con tutte le proprietà
            query = f"""
            MERGE (e:{categoria} {{nome: $nome}})
            SET e += $properties
            """
            
            try:
                # Esegui la query
                session.run(query, nome=nome_decodificato, properties=properties)
                print(f"Caricata entità {nome_decodificato} in categoria {categoria}")
            except Exception as e:
                print(f"Errore nel caricamento dell'entità {nome_decodificato} in categoria {categoria}: {e}")
        
        print(f"Entità della categoria {categoria} caricate correttamente in Neo4j.")

## MAIN

In [ ]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo

# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo

# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)
# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)

In [ ]:
with open('entita_filtrate.txt', 'r', encoding='utf-8') as file:
    entita_filtrate = [line.strip() for line in file if line.strip()]

print(entita_filtrate)

['Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Piatto', 'Ristorante', 'Pianeta']


In [ ]:
categorie = estrai_categorie_rilevanti(testo_pianeti, entita_filtrate, llm)
print(categorie)
for categoria in categorie:
    valorizzazioni = estrai_valorizzazioni(testo_pianeti, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

['Pianeta']
Caricata entità Tatooine in categoria Pianeta
Caricata entità Asgard in categoria Pianeta
Caricata entità Namecc in categoria Pianeta
Caricata entità Arrakis in categoria Pianeta
Caricata entità Krypton in categoria Pianeta
Caricata entità Pandora in categoria Pianeta
Caricata entità Cybertron in categoria Pianeta
Caricata entità Ego in categoria Pianeta
Caricata entità Montressosr in categoria Pianeta
Caricata entità Klyntar in categoria Pianeta
Entità della categoria Pianeta caricate correttamente in Neo4j.


In [150]:
categorie = estrai_categorie_rilevanti(testo_manuale, entita_filtrate, llm)
print(categorie)
for categoria in categorie:
    valorizzazioni = estrai_valorizzazioni(testo_manuale, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

['Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Piatto', 'Ristorante', 'Pianeta']
Entità senza nome trovata in categoria Tecnica, skip
Entità della categoria Tecnica caricate correttamente in Neo4j.
Caricata entità Psionica in categoria Licenza
Caricata entità Temporale in categoria Licenza
Caricata entità Gravitazionale in categoria Licenza
Caricata entità Antimateria in categoria Licenza
Caricata entità Magnetica in categoria Licenza
Caricata entità Quantistica in categoria Licenza
Caricata entità Luce in categoria Licenza
Caricata entità Livello di Sviluppo Tecnologico (LTK) in categoria Licenza
Entità della categoria Licenza caricate correttamente in Neo4j.
Caricata entità Ordine della Galassia di Andromeda in categoria Ordine
Caricata entità Ordine dei Naturalisti in categoria Ordine
Caricata entità Ordine degli Armonisti in categoria Ordine
Entità della categoria Ordine caricate correttamente in Neo4j.
Entità senza nome trovata in categoria Ingrediente, skip
Entità della categor

In [151]:
categorie = estrai_categorie_rilevanti(testo_codice, entita_filtrate, llm)
print(categorie)
for categoria in categorie:
    valorizzazioni = estrai_valorizzazioni(testo_codice, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

['Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Ristorante', 'Pianeta']
Errore nel caricamento dell'entità Marinatura a infusione gravitazionale in categoria Tecnica: {code: Neo.ClientError.Statement.TypeError} {message: Property values can only be of primitive types or arrays thereof. Encountered: Map{Tipo -> String("gravitazionale (G)"), Livello -> String("II")}.}
Errore nel caricamento dell'entità Marinatura temporale sincronizzata in categoria Tecnica: {code: Neo.ClientError.Statement.TypeError} {message: Property values can only be of primitive types or arrays thereof. Encountered: Map{Tipo -> String("temporale (t)"), Livello -> String("I")}.}
Errore nel caricamento dell'entità Marinatura psionica in categoria Tecnica: {code: Neo.ClientError.Statement.TypeError} {message: Property values can only be of primitive types or arrays thereof. Encountered: Map{Tipo -> String("psionica (P)"), Livello -> String("III")}.}
Errore nel caricamento dell'entità Marinatura tramite reazioni d’ant

In [ ]:
categorie = estrai_categorie_rilevanti(testo_blog1, entita_filtrate, llm)
print(categorie)
for categoria in categorie:
    valorizzazioni = estrai_valorizzazioni(testo_blog1, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

In [ ]:
categorie = estrai_categorie_rilevanti(testo_blog2, entita_filtrate, llm)
print(categorie)
for categoria in categorie:
    valorizzazioni = estrai_valorizzazioni(testo_blog2, categoria, llm, driver)
    carica_entita_su_neo4j(categoria, valorizzazioni, driver)

In [ ]:
menu_esempio = "../Hackapizza Dataset/Menu/Anima Cosmica.pdf"
testo_menu = estrai_testo_pdf(menu_esempio)

# ESTRAI DA CODICE
for categoria in entita_filtrate: 
    valori = estrai_valorizzazioni(testo_menu, categoria, llm, driver)
    carica_entita_su_neo4j(entities=valori, categoria=categoria, driver=driver)

